In [1]:
import pathlib

import geodatasets

# geopandas map of the USA
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim

In [2]:
collaborators_file_path = pathlib.Path("../data/Way_Lab_Collaborators.csv")
df = pd.read_csv(collaborators_file_path)
df.head()

,Name,Organization,Projects,First collaboration,Country,State,City
0,Jim Walker,Massachusetts General Hospital,NF1 CP,2022,United States,Massachusetts,Boston
1,Michelle Mattson-Hoss,Infixion Bioscience,NF1 CP,2022,United States,California,San Diego
2,Jaime Cheah,Broad Institute,ALSF/NF1 CP,2023,United States,Massachusetts,Boston
3,Jackie Taroni,Alex's Lemonade Stand,ALSF,2023,United States,Philadelphia,Pennsylvania
4,Tim McKinsey,University of Colorado,CFReT,2022,United States,Colorado,Denver


In [3]:
# get the latitude and longitude of each city, state, country combination
# using the geopy library

geolocator = Nominatim(user_agent="http")

# initialize the location column
df["location"] = df["City"] + ", " + df["State"] + ", " + df["Country"]
df["latitude"] = np.nan
df["longitude"] = np.nan

for i in range(len(df)):
    try:
        location = geolocator.geocode(df["location"][i], timeout=100)
        df.loc[i, "latitude"] = location.latitude
        df.loc[i, "longitude"] = location.longitude
    except:
        print(f"Error: {df['location'][i]} not found")

In [4]:
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326"
)
gdf.head()

,Name,Organization,Projects,First collaboration,Country,State,City,location,latitude,longitude,geometry
0,Jim Walker,Massachusetts General Hospital,NF1 CP,2022,United States,Massachusetts,Boston,"Boston, Massachusetts, United States",42.355433,-71.060511,POINT (-71.06051 42.35543)
1,Michelle Mattson-Hoss,Infixion Bioscience,NF1 CP,2022,United States,California,San Diego,"San Diego, California, United States",32.717420,-117.162772,POINT (-117.16277 32.71742)
2,Jaime Cheah,Broad Institute,ALSF/NF1 CP,2023,United States,Massachusetts,Boston,"Boston, Massachusetts, United States",42.355433,-71.060511,POINT (-71.06051 42.35543)
3,Jackie Taroni,Alex's Lemonade Stand,ALSF,2023,United States,Philadelphia,Pennsylvania,"Pennsylvania, Philadelphia, United States",39.950394,-75.194671,POINT (-75.19467 39.95039)
4,Tim McKinsey,University of Colorado,CFReT,2022,United States,Colorado,Denver,"Denver, Colorado, United States",39.739236,-104.984862,POINT (-104.98486 39.73924)


In [5]:
# save the geopandas dataframe to a shapefile
gdf.to_file("../data/collaborator_locations.shp")

/tmp/ipykernel_379223/1563633796.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("../data/collaborator_locations.shp")
/home/lippincm/miniforge3/envs/mapping_env/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Organization' to 'Organizati'
  ogr_write(
/home/lippincm/miniforge3/envs/mapping_env/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'First collaboration' to 'First coll'
  ogr_write(
